In [37]:
import ast

In [47]:
import re, copy
def get_type(node):
    return node.__class__.__name__
    return re.sub(r"^<class 'ast\.(.+)'>$", r'\1', str(type(node)))
def eval_binop(node, variables):
    if get_type(node) == "Constant":
        return node

    if get_type(node) == "Name":
        #return variables[node.id]
        #ret = ast.Constant(variables[node.id])
        ret = ast.Constant(variables[node.id])
        return ret
    
    elif get_type(node) == "UnaryOp":
        cp = copy.deepcopy(node)
        cp.operand = eval_binop(cp.operand, variables)
        
        expr = ast.Expression(body=cp)
        ast.fix_missing_locations(expr)
        exe = compile(expr, filename="", mode="eval")
        return ast.Constant(eval(exe))
    
    elif get_type(node) in ("BinOp"):
        cp = copy.deepcopy(node)
        cp.left = eval_binop(cp.left, variables)
        cp.right = eval_binop(cp.right, variables)
        
        expr = ast.Expression(body=cp)
        ast.fix_missing_locations(expr)
        exe = compile(expr, filename="", mode="eval")
        return ast.Constant(eval(exe))
    
    elif get_type(node) in ("BoolOp"):
        cp = copy.deepcopy(node)
        for i in range(len(cp.values)):
            cp.values[i] = eval_binop(cp.values[i], variables)
        expr = ast.Expression(body=cp)
        ast.fix_missing_locations(expr)
        exe = compile(expr, filename="", mode="eval")
        return ast.Constant(eval(exe))
    
    elif get_type(node) in ("Compare"):
        cp = copy.deepcopy(node)
        cp.left = eval_binop(cp.left, variables)
        for i in range(len(cp.comparators)):
            cp.comparators[i] = eval_binop(cp.comparators[i], variables)
        expr = ast.Expression(body=cp)
        ast.fix_missing_locations(expr)
        exe = compile(expr, filename="", mode="eval")
        return ast.Constant(eval(exe))

In [41]:
with open("./mysite/tmp.py", 'r', encoding='utf-8') as f:
    kod = f.read()

print(kod)
node = ast.parse(kod)
for x in ast.iter_child_nodes(node):
    print(x)

a = 3

b = True or False
c= None


In [42]:
def get_exprs(node):
    ret = []
    #if isinstance(node, ast.expr): #TODO ast.expr
    if node.__class__ in [ast.BinOp, ast.BoolOp, ast.Compare, ast.UnaryOp]:
    #if isinstance(node, ast.BinOp) or isinstance(node, ast.BoolOp) or isinstance(node, ast.Compare) or isinstance(node, ast.UnaryOp):
        ret.append(node)
    else:
        for child in ast.iter_child_nodes(node):
            ret = ret + get_exprs(child)
    return ret
exprs = get_exprs(node)
for e in exprs:
    print(e, e.lineno)

<_ast.BoolOp object at 0x7f13507304c0> 3


In [43]:

bo = exprs[0]
tree = node2tree(bo, kod.split('\n'), {'z':99})
'''
print(ast.dump(bo.right))
print(bo.right.__dict__)
'''
'''
ret = eval_binop(bo, {'z':99})
ast.dump(ret)

tree = node2tree(bo, kod.split('\n'), {'z':99})

tree
'''

"\nret = eval_binop(bo, {'z':99})\nast.dump(ret)\n\ntree = node2tree(bo, kod.split('\n'), {'z':99})\n\ntree\n"

In [49]:
tree = node2tree(bo, kod.split('\n'), {'z':99})
tree

ez a type BoolOp
ez a type Or
ez a type Constant
ez a type Constant


{'type': 'BoolOp',
 'from_line': 2,
 'to_line': 2,
 'from_char': 4,
 'to_char': 16,
 'code': 'True or False',
 'eval': True,
 'children': [{'type': 'Or', 'eval': None, 'children': []},
  {'type': 'Constant',
   'from_line': 2,
   'to_line': 2,
   'from_char': 4,
   'to_char': 7,
   'code': 'True',
   'eval': True,
   'children': []},
  {'type': 'Constant',
   'from_line': 2,
   'to_line': 2,
   'from_char': 12,
   'to_char': 16,
   'code': 'False',
   'eval': False,
   'children': []}]}

In [50]:

def tree2seq(node, start=True):
    #ret = []
    if start:
        #deleting duplicates
        lst = tree2seq(node, False)
        ret = [lst[0]]
        for x in lst[1:]:
            if x != ret[-1]:
                ret.append(x)
        return ret
    
    if node['type'] in ("Constant"):
        return [node['code']]
    elif node['type'] in ("Name", "UnaryOp"):
        return [node['code'], str(node['eval'])]
    elif node['type'] in ("BinOp", "BoolOp", "Compare"):
        
        if node['type'] in ("BinOp", "Compare"):
            left_child = node['children'][0]
            right_child = node['children'][2]
        elif node['type'] == "BoolOp":
            left_child = node['children'][1]
            right_child = node['children'][2]
        
        left_from = left_child['from_char'] - node['from_char']
        left_to = left_from + len(left_child['code'])
        right_from = right_child['from_char'] - node['from_char']
        right_to = right_from + len(right_child['code'])
           
        
        first_part = node['code'][:left_from]
        middle_part = node['code'][left_to:right_from]
        last_part = node['code'][right_to:]
        
        
        
        ret = [node['code']]
        
        left_seq = tree2seq(left_child, False)
        #print(left_seq)
        for left_step in left_seq:
            sb = first_part
            sb += left_step
            sb += middle_part
            sb += right_child['code']
            sb += last_part
            ret.append(sb)
            
        right_seq = tree2seq(right_child, False)
        for right_step in right_seq:
            sb = first_part
            sb += left_seq[-1]
            sb += middle_part
            sb += right_step
            sb += last_part
            ret.append(sb)
        
        ret.append(str(node['eval']))
        
        return ret
    
seq = tree2seq(tree)
for x in seq:
    print(x)
#print(tree)

True or False
True


In [48]:
def node2tree(node, kod_mtx, variables):
	d = dict()
	d['type'] = get_type(node)
	print("ez a type", d['type'])
	
	# if d['type'] not in ['Module', 'Store', 'Add', 'Mult', 'Div']:
	if 'lineno' in node.__dict__:
		
		d['from_line'] = node.lineno-1
		d['to_line'] = node.end_lineno-1
		d['from_char'] = node.col_offset
		d['to_char'] = node.end_col_offset-1
		#ha egy sorban van az egész node
		if node.lineno == node.end_lineno:
			d['code'] = kod_mtx[node.lineno-1][node.col_offset:node.end_col_offset]
		#ha két sorban van az egész node
		elif node.lineno + 1 == node.end_lineno:
			d['code'] = kod_mtx[node.lineno-1][node.col_offset:]
			d['code'] += kod_mtx[node.end_lineno-1][:node.end_col_offset]
		#ha több, mint két sorban van a node
		elif node.lineno + 1 < node.end_lineno:
			d['code'] = kod_mtx[node.lineno-1][node.col_offset:]
			for i in range(node.lineno, node.end_lineno-1):
				d['code'] += kod_mtx[i]
			d['code'] += kod_mtx[node.end_lineno-1][:node.end_col_offset]
		else:
			raise Exception("baj a kód kiolvasással")
	
	if d['type'] in ("BinOp", "BoolOp", "Compare", "UnaryOp"):
		d['eval'] = eval_binop(node, variables).value
	elif d['type'] == "Name":
		#d['eval'] = eval_binop(variables[node.id], variables).value
		d['eval'] = eval_binop(node, variables).value        
	elif d['type'] == "Constant":
		d['eval'] = node.value
	else:
		d['eval'] = None
	
	if d['type'] == "Assign":
		targets = node.targets
		t = targets[0].id
		variables[t] = eval_binop(node.value)
		print(variables)
		
	
	d['children'] = [node2tree(child, kod_mtx, variables) for child in ast.iter_child_nodes(node)]
	return d

In [57]:
def to_html(text):
	return text.replace('\n', ';').replace('\t', '	')

def to_treant(node):
	ret = dict()
	ret['text'] = {'name': node['type']}
	if 'eval' in node:
		ret['text']['title'] = node['eval'] if node['eval'] else ''
	if 'code' in node:
		ret['text']['desc'] = to_html(node['code'])
	ret['children'] = [to_treant(child) for child in node['children']]
	return ret

In [59]:
to_treant(tree)

{'text': {'name': 'BoolOp', 'title': '', 'desc': 'not True or 2<4 and z == 2'},
 'children': [{'text': {'name': 'Or', 'title': ''}, 'children': []},
  {'text': {'name': 'UnaryOp', 'title': '', 'desc': 'not True'},
   'children': [{'text': {'name': 'Not', 'title': ''}, 'children': []},
    {'text': {'name': 'Constant', 'title': True, 'desc': 'True'},
     'children': []}]},
  {'text': {'name': 'BoolOp', 'title': '', 'desc': '2<4 and z == 2'},
   'children': [{'text': {'name': 'And', 'title': ''}, 'children': []},
    {'text': {'name': 'Compare', 'title': True, 'desc': '2<4'},
     'children': [{'text': {'name': 'Constant', 'title': 2, 'desc': '2'},
       'children': []},
      {'text': {'name': 'Lt', 'title': ''}, 'children': []},
      {'text': {'name': 'Constant', 'title': 4, 'desc': '4'},
       'children': []}]},
    {'text': {'name': 'Compare', 'title': '', 'desc': 'z == 2'},
     'children': [{'text': {'name': 'Name', 'title': 99, 'desc': 'z'},
       'children': [{'text': {'name